# Classification binaire


On classifie des critiques de film en deux catégories: "critique positive" et  "critique négative".


In [ ]:
%reset -f

In [ ]:
from tensorflow import keras

import jax
import jax.numpy as jnp
import jax.random as jr
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import optax
import pickle
from dataclasses import dataclass
import time
from typing import Callable
import os
import shutil

## Data

###  IMDB dataset




On télécharge le jeu de donnée IMBD.
* Les data sont des critiques de films
* les labels des entier 0 ou 1 indiquant si la critique est négative ou positive







In [ ]:
num_words=10_000

In [ ]:
"download the dataset (80MB): it is done only the first time"
from keras.datasets import imdb
(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=num_words)

ci-dessus, `num_words=10000` signifie que l'on n'a gardé que les 10000 mots les plus fréquents. Les autres ont été supprimés.

In [ ]:
#Il y a 50_000 critiques en tout
len(train_data),len(test_data)

In [ ]:
n_val=15_000
data={"train":train_data,"val":test_data[:n_val],"test":test_data[n_val:]}
labels={"train":train_labels,"val":test_labels[:n_val],"test":test_labels[n_val:]}

Les data: Chaque élément est une liste d'indices. Chaque indice représente un mot

In [ ]:
#les review n'ont pas toute la même longueur
data["train"]

In [ ]:
#affichons les longueurs des 10 premières phrases
for i in range(10):
    print(len(data["train"][i]))

***A vous:*** Toutes le listes commencent par 1. Pourquoi ? Le 0 est-il utilisé ?

Voici l'histogramme des longeurs de critiques.

In [ ]:
length_sentences=[len(sentence) for sentence in data["train"]]
val,count=np.unique(length_sentences,return_counts=True)
fig,ax=plt.subplots(figsize=(15,2))
ax.bar(val,count);

Les deux classes sont équilibrées

In [ ]:
labels["train"]

In [ ]:
val,count=np.unique(labels["train"],return_counts=True)
plt.bar(val,count);

***A vous:*** C'est quoi ce 9999?

In [ ]:
max([max(sequence) for sequence in data["train"]])

***A vous:*** Faite l'histogramme des tokens (=mots traduits en entier) pour voir quels sont les mots les plus fréquents.

### Décodons


Transformons les indices en mots:

In [ ]:
# word_index is a dictionary mapping words to an integer index
word_index = imdb.get_word_index()
# We reverse it, mapping integer indices to words
reverse_word_index = {value:key for (key, value) in word_index.items()}
# We decode the review; note that our indices were offset by 3
# because 0, 1 and 2 are reserved indices for "padding", "start of sequence", and "unknown".
decoded_review = ' '.join([reverse_word_index.get(i - 3, '£') for i in train_data[0]])

In [ ]:
decoded_review

***A vous:*** Cette review est clairement positive. Cherchez-en une négative et décodez là $(2\heartsuit)$.

### Encodage des données


Des indices représentant des mots sont des variables qualitatives. Il faut les numériser. Deux techniques existes:

* "vord2vec" il s'agit de représenter chaque mot par un vecteur de grande dimension, de telle manière à ce que les relations sémentiques entre les mots se traduisent en relation vectorielles. Verra cela plus tard.

* "one_hot_encoding": une review du type `[3, 5 ,1]` sera changée un vecteur de taille 10 000 composés de 0 sauf pour les indices 3,5,1 qui seront mis à 1.


In [ ]:
def encode_sequences(sequences, dimension):
    # Create an all-zero matrix of shape (len(sequences), dimension)
    results = np.zeros((len(sequences), dimension), dtype=np.int32)
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1  # set specific indices of results[i] to 1s
    return jnp.array(results) #passage sur GPU

***A vous:*** $(1\heartsuit)$ Est-ce l'ordre des mots a une importance après l'encodage?  Est-ce que la répétition d'un même mot a une importance?



***A vous:*** ($2\heartsuit$) implantez une fonction `encode_sequences_with_count` qui prend en compte le nombre d'apparition des mots. Par exemple:

    encode_sequences_with_count([[3,1,1,3],[1,2,2]],10)
    
renvera

    [[0 2 0 2 0 0 0 0 0 0]
    [0 1 2 0 0 0 0 0 0 0]]

In [ ]:
for key in ["train","val","test"]:
    data[key]=encode_sequences(data[key],num_words)

In [ ]:
data["train"].shape

Encodate des outputs:

1. On les transformes en float et on les met en jax (donc sur GPU s'il y en a un)

2. On les mets au format data frame




In [ ]:
for key in ["train","val","test"]:
    labels[key]=jnp.array(labels[key],dtype=jnp.float32)[:,None]

In [ ]:
labels["train"].shape

Le points 1. n'est pas obligatoire mais améliore les performance: ces labels seront utilisés de nombreuses fois. Si on ne fait pas ces opérations, elles seront faites automatiquement à chaque calcul de loss

Le points 2 est crucial. Voir plus bas.

### Distributeur de batch

In [ ]:
"""  distributeur de donnée par batch.   """
def oneEpoch(X_all,Y_all,batch_size):

    nb_batches=len(X_all)//batch_size

    shuffle_index=np.random.permutation(len(X_all))
    X_all_shuffle=X_all[shuffle_index]
    Y_all_shuffle=Y_all[shuffle_index]

    for i in range(nb_batches):
        yield X_all_shuffle[i*batch_size:(i+1)*batch_size],Y_all_shuffle[i*batch_size:(i+1)*batch_size]

In [ ]:
for x,y in oneEpoch(data["train"],labels["train"],256):
    print(x.shape)
    print(y.shape)
    break

## Model

###  mathématiquement


1/ nous créons un modèle: il s'agissait d'une fonction $x \to model_w(x)$ à valeur dans $[0,1]$, paramétrée par $w$. Cette fonction est la composition de fonctions très simple:

* Fonction linéaire
* Fonction `relu` (pour introduire des non-linéarité)
* Fonction sigmoide (pour fini dans $[0,1]$)


2/ Notre intuition est la suivante: Pour un certain paramètre $w$,  pour tout couple de données $(x,y)$, on aura:
$$
model_w(x)= \hat y\in[0,1]  \qquad \text{ est proche de } \qquad  y \in \{0,1\}
$$

3/ On choisit une 'distance' (une loss) pour mesure l'écart entre $\hat y$ et  $y$, c'est la crossentropy **binaire**:
$$
\mathtt{BCE}(y, \hat y ) :=  - y \log (\hat y )   -  (1-y) \log(1-\hat y)
$$
Que l'on somme sur toutes les observations
$$
loss_w=\sum_{i \in Batch} \mathtt{BCE} (y_i, \hat y_i )= \sum_{i \in Batch} \mathtt{BCE} (y_i, model_w(x_i) )
$$
On demande à l'algo d'optimisation de trouver le $\hat w$ qui minimise cette loss sur plein de batchs.

4/ La fonction $x\to model_{\hat w} (x)$ sera un bon outil pour prédire des nouveaux input $x$.






### Construction

On va construire un réseau dense:

In [ ]:
def model_fnm(hyper_param):

    dim_in=10_000
    dim_out=1
    n_layer=hyper_param["n_layer"]
    dim_hidden=hyper_param["dim_hidden"]


    layer_widths=[dim_in] + [dim_hidden]*(n_layer-1) + [dim_out]


    def model_init(rkey):
        params = []
        for n_in, n_out in zip(layer_widths[:-1], layer_widths[1:]):
            rk,rkey=jr.split(rkey)
            params.append(
                {"weight":jr.normal(rk,shape=(n_in, n_out))*jnp.sqrt(2/n_in),
                "bias":jnp.zeros([n_out])})
        return params


    @jax.jit
    def model_apply(params, inp):
        *hidden, last = params
        for layer in hidden:
            inp = jax.nn.relu(inp @ layer['weight'] + layer['bias'])

        #classification binaire: on ressort une proba
        return jax.nn.sigmoid(inp @ last['weight'] + last['bias'])

    return model_init,model_apply


***Remarque:*** Pour chaque couche cachée: le nombre de neurone (=la dimension=nb units) correspond à "combien de degré de liberté on donne au réseau de neurone pour se représenter le problème". Avec beaucoup de liberté, le réseau peut se faire des représentations plus complexe. Mais cela requiert plus de calcul, et il peut apprendre des motifs (=pattern) superflus: propre au donnée train et donc non généralisable.


## Tout pour l'entrainement

### Loss et accuracy

In [ ]:
def binary_cross_entropy(y_true,y_pred):
    epsilon=1e-6
    y_pred=jnp.clip(y_pred,epsilon,1-epsilon)
    return jnp.mean(-y_true*jnp.log(y_pred)-(1-y_true)*jnp.log(1-y_pred))

In [ ]:
batch_size=7

Une bonne loss

In [ ]:
y_true=jnp.ones([7,1])
y_pred=jnp.ones([7,1])*0.99
binary_cross_entropy(y_true,y_pred)

Une mauvaise loss

In [ ]:
y_true=jnp.ones([7,1])
y_pred=jnp.ones([7,1])*0.1
binary_cross_entropy(y_true,y_pred)

Une loss bof bof

In [ ]:
y_true=jnp.ones([7,1])
y_pred=jnp.ones([7,1])*0.5
binary_cross_entropy(y_true,y_pred)

***A vous:*** ♡♡♡ Créez une dataframe aléatoire `y_pred` avec des valeurs uniformes sur [0,1]. Calculez la loss entre ces prédiction est un vecteur constitudé de 0 ou de 1 (peut importe).

A quelle est la loss attendue ? Vérifiez

### Le bug hyper-classique

In [ ]:
batch_size=20
y_true=jr.choice(jr.key(0),a=jnp.array([0,1]),shape=[batch_size])
y_true

Imaginons des prédictions parfaites, mais présentée sous la forme d'une dataframe (ici une matrice colonne). C'est naturel car les réseaux ne neurones qui prennent en entrée des dataframes renvoie des dataframes.

In [ ]:
y_pred=y_true[:,None]

In [ ]:
binary_cross_entropy(y_true,y_pred)

***A vous:*** Expliquer pourquoi l'on n'a pas une loss très petite. Retenez bien ce conseille: il faut toujours transformer input et output en dataframe (même quand la dimension est 1).

### La précision (accuracy)

In [ ]:
def accuracy(y_true,y_pred):
    y_pred=jnp.round(y_pred).astype(bool)
    y_true=y_true.astype(bool)
    return jnp.mean(y_pred==y_true)

In [ ]:
y_true=jnp.ones([7,1])
y_pred=jnp.ones([7,1])*0.51
accuracy(y_true,y_pred)

In [ ]:
y_true=jnp.ones([7,1])
y_pred=jnp.ones([7,1])*0.49
accuracy(y_true,y_pred)


***A vous:*** Pourquoi n'est-il pas bon  d'utiliser 1-accuracy comme loss ?





###  L'update des params

In [ ]:
def jit_creator(hyper_param):


    model_init, model_apply = model_fnm(hyper_param)
    optimizer = optax.adam(hyper_param["learning_rate"])


    @jax.jit
    def loss_compute(params, X,Y_true):
        Y_pred=model_apply(params,X)
        return binary_cross_entropy(Y_true,Y_pred)


    @jax.jit
    def accuracy_compute(params, X,Y_true):
        Y_pred=model_apply(params,X)
        return accuracy(Y_true,Y_pred)


    @jax.jit
    def update_model_param(optimizer_state, model_param, X,Y_true):
        loss,grads = jax.value_and_grad(loss_compute)(model_param, X,Y_true )
        updates, optimizer_state = optimizer.update(grads, optimizer_state)
        #here the model_param is modified
        model_param = optax.apply_updates(model_param, updates)
        return loss,optimizer_state, model_param


    return model_init, model_apply, optimizer, loss_compute,accuracy_compute,update_model_param

### Fonctions de sauvegarde

In [ ]:
def save_as_pickle(file_name,serializable):
    pickle.dump(serializable,open(file_name,"wb"))
def load_from_pickle(file_name):
    return pickle.load(open(file_name,"rb"))
def save_as_str(file_name,serializable):
    with open(file_name, "wt") as f:
        f.write(str(serializable))
def load_from_str(file_name):
    with open(file_name, "rt") as f:
        res = eval(f.read())
    return res

### L'Agent en personne

In [ ]:
@dataclass
class AgentResult:
    hyper_param:dict
    best_loss:float
    accuracy_at_best:float
    model_param:dict
    model_apply:Callable


class Agent:
    @staticmethod
    def load(folder):
        assert os.path.exists(folder),f"folder:{folder} does not exist"
        model_param = load_from_pickle(f"{folder}/model_param")
        best_loss = load_from_str(f"{folder}/best_loss")
        accuracy_at_best = load_from_str(f"{folder}/accuracy_at_best")
        model_param=load_from_pickle(f"{folder}/model_param")
        hyper_param=load_from_str(f"{folder}/hyper_param")
        _,model_apply=model_fnm(hyper_param)
        return AgentResult(hyper_param,best_loss,accuracy_at_best,model_param,model_apply)



    @staticmethod
    def train(folder,hyper_param,jit_creator,n_epoch,verbose=True):

        #on repart de zéro
        shutil.rmtree(folder,ignore_errors=True)
        os.makedirs(folder)

        losses=[]
        val_losses=[]
        val_steps=[]


        model_init, model_apply, optimizer, loss_compute, accuracy_compute, update_model_param=jit_creator(hyper_param)




        batch_size = hyper_param["batch_size"]



        if verbose:
            print(f"New folder:{folder}, model_param are randomly initialized")
        model_param=model_init(jr.key(0))

        best_loss=1e10#l'infini ou presque


        save_as_str(f"{folder}/hyper_param", hyper_param)
        optimizer_state=optimizer.init(model_param)

        step=0

        for _ in range(n_epoch):

            for x,y in oneEpoch(data["train"],labels["train"],batch_size):
                step+=1
                loss,optimizer_state, model_param = update_model_param(optimizer_state, model_param, x,y)
                losses.append(loss)



            val_loss=loss_compute(model_param, data["val"], labels["val"])
            val_steps.append(step)
            val_losses.append(val_loss)


            if val_loss <= best_loss:
                best_loss=val_loss
                accuracy_at_best=accuracy_compute(model_param, data["val"], labels["val"])

                save_as_pickle(f"{folder}/model_param",model_param)
                save_as_str(f"{folder}/best_loss",best_loss)
                save_as_str(f"{folder}/accuracy_at_best",accuracy_at_best)

                if verbose:
                        print(f"⬊{val_loss:.3g}", end="")

            else:
                if verbose:
                    print(".",end="")
        if verbose:
            print("| end of the optimization loop.")

        return losses,val_losses,val_steps


## Entrainement

### Premier entrainement

In [ ]:
folder="model_normal"

hyper_param={"learning_rate":1e-3,"batch_size":256,"n_layer":2,"dim_hidden":20}

losses,val_losses,val_steps=Agent.train(folder,hyper_param,jit_creator,10)

In [ ]:
plt.plot(losses,label="loss")
plt.plot(val_steps,val_losses,".",label="val_loss")
plt.legend();


* la loss décroit sans arrêt: notre optimizer fonctionne.
* Mais la val_loss remonte après 2 ou 3 epochs

C'est typique du sur-apprentissage (over-fitting): L'optimizer apprend des motifs spécifique au donnée train et donc qui ne se généralise pas au données `val`, et donc pas non plus aux données `test` et au futur données entrantes.





### Evaluons le modèle sur les données tests

In [ ]:
folder="model_normal"
agent_result=Agent.load(folder)

In [ ]:
Y_pred=agent_result.model_apply(agent_result.model_param,data["test"])

In [ ]:
accuracy(labels["test"],Y_pred)

89% d'accuracy: pas mal pour un modèle aussi simple.

***A vous:*** $(2\heartsuit)$  Pour le protocole que l'on a utilisé:  était-il vraiment nécessaire d'avoir des données `validation` et `test` distinctes?

### Annalyse de prédiction


In [ ]:
Y_pred_cat=(Y_pred>0.5)[:,0].astype(int)
Y_pred_cat

In [ ]:
Y_true_cat=labels["test"][:,0].astype(int)
Y_true_cat

***A vous:*** Observez quelques critiques mal classifiées. Décodez la en anglais pour voir si elles étaients ambigües.

In [ ]:
fig,(ax0,ax1)=plt.subplots(1,2,figsize=(8,2))
ax0.hist(Y_pred[Y_true_cat==0],bins=40,edgecolor="k")
ax1.hist(Y_pred[Y_true_cat==1],bins=40,edgecolor="k");

On peut voir que le réseau est assez confiant en ses prédictions: la majorité des probas est proche de 0 ou de 1.


###Exo: Faites vos propres expérimentation

* $(3\heartsuit)$ essayez en vectorisant les séquences avec `vectorize_sequences_with_count`
* $(3\heartsuit)$ Essayer d'autre architecture
* $(3\heartsuit)$ Essayez la loss `mse` à la place de la cross-entropy-binaire. Ce n'est pas classique, mais cela fonctionne aussi.
* $(3\heartsuit)$ Essayez d'autre fonctions d'activations dans les couches cachées.

## Lutter contre le sur-apprentissage







### Réduire la taille du modèle

In [ ]:
folder="model_small"

hyper_param={"learning_rate":1e-3,"batch_size":256,"n_layer":2,"dim_hidden":8}

losses,val_losses,val_steps=Agent.train(folder,hyper_param,jit_creator,10)

In [ ]:
plt.plot(losses,label="loss")
plt.plot(val_steps,val_losses,".",label="val_loss")
plt.legend();

In [ ]:

folder="model_big"

hyper_param={"learning_rate":1e-3,"batch_size":256,"n_layer":2,"dim_hidden":256}

losses,val_losses,val_steps=Agent.train(folder,hyper_param,jit_creator,10)

In [ ]:
plt.plot(losses,label="loss")
plt.plot(val_steps,val_losses,".",label="val_loss")
plt.legend();

⇑ Plus le modèle est gros, et plus vite arrive le sur-apprentissage

### Pénaliser




  _Occam's Razor_ principle: étant donnée 2 explications valables,  la meilleure est la plus simple.
   

![toot](https://drive.google.com/uc?export=view&id=19UwF49NvDNwl7NlyTkn_5WZl1bF9AL5P)


Cela s'applique aussi au modèles de machine learning: Un modèle simple dans ce contexte, est celui dont la distribution des paramètre la moins d'entropy. Pour  régulariser on peut forcer les poids de prendre des petites valeurs, ce qui rend la distribution des poids plus régulière. C'est la   "weight regularization" ou "pénalisation des poids".

Pour ce faire on ajouter un terme à la loss, qui est grand quand les poids sont grands en valeur absolue.
Les deux techniques principales sont:

* Régularisation L1 ou lasso:
$$
loss_\alpha= loss + \alpha \sum_i |w_i|
$$
* Régularisation L2 ou ridge:
$$
loss_\alpha= loss +\alpha \sum_i (w_i)^2
$$
La somme étant faite sur tous les poids $w_i$ appartenant à un 'kernel' (une matrice).


Remarquons qu'on ne pénalise pas en général les biais.

Le travail des biais est de recentrer les données autour de zéro à chaque couche (0 c'est la partie intéressante des fonctions d'activations): il faut donc laisser les bias faire leur travail sans les pénaliser.

In [ ]:
def jit_creator_with_penalization(hyper_param):


    model_init, model_apply = model_fnm(hyper_param)
    optimizer = optax.adam(hyper_param["learning_rate"])


    #la loss non pénaliser sera encore utiliser pour la validation
    @jax.jit
    def loss_compute(params, X,Y_true):
        Y_pred=model_apply(params,X)
        return binary_cross_entropy(Y_true,Y_pred)


    positive_fn=jnp.square if hyper_param["penalization_type"]=="l2" else jnp.abs
    @jax.jit
    def penalization_compute(params):
        res=jnp.array(0.)
        for p in jax.tree.leaves(params):
            res+=jnp.sum(positive_fn(p))
        return res


    @jax.jit
    def loss_compute_plus_penalization(params, X,Y_true):
        return loss_compute(params, X,Y_true)+ hyper_param["penalization_coef"]*penalization_compute(params)


    @jax.jit
    def accuracy_compute(params, X,Y_true):
        Y_pred=model_apply(params,X)
        return accuracy(Y_true,Y_pred)


    @jax.jit
    def update_model_param(optimizer_state, model_param, X,Y_true):
        loss,grads = jax.value_and_grad(loss_compute_plus_penalization)(model_param, X,Y_true )
        updates, optimizer_state = optimizer.update(grads, optimizer_state)
        #here the model_param is modified
        model_param = optax.apply_updates(model_param, updates)
        return loss,optimizer_state, model_param


    return model_init, model_apply, optimizer, loss_compute,accuracy_compute,update_model_param

Pourquoi est-ce qu'il ne faut pas pénaliser la loss de validation: je reprend une analofie de grok:

Imagine un athlète :

* Entraînement : il porte un sac à dos de 10 kg (→ régularisation)

* Compétition : on enlève le sac (→ évaluation réelle)

Tu ne juges pas sa performance avec le sac !

In [ ]:
folder="model_normal_with_penalization_l2"

hyper_param={"learning_rate":1e-3,"batch_size":256,"n_layer":2,"dim_hidden":20,
             "penalization_type":"l2", "penalization_coef":1e-3}

losses,val_losses,val_steps=Agent.train(folder,hyper_param,jit_creator_with_penalization,10)

In [ ]:
plt.plot(losses,label="loss")
plt.plot(val_steps,val_losses,".",label="val_loss")
plt.legend();

In [ ]:
from jax.flatten_util import ravel_pytree


def compare_weight_distribution(folders):
    ni=len(folders)
    fig,axs=plt.subplots(ni,1,figsize=(4,4*ni))
    if ni==1:
        axs=[axs]

    bins=jnp.linspace(-0.15,0.15,30)
    for i,folder in enumerate(folders):
        agent_result=Agent.load(folder)
        params=agent_result.model_param
        #on ne garder que les kernels
        params=[p for p in jax.tree.leaves(params) if len(p.shape)==2]

        params_flat, unflatten_fn = ravel_pytree(params)
        axs[i].hist(params_flat,bins=bins)
        axs[i].set_title(folder)

    fig.tight_layout()

compare_weight_distribution(["model_normal","model_normal_with_penalization_l2"])

On doit bien ajuster le coefficient de pénalisation $\alpha$:

* $\alpha$ trop petit la pénalisation est inefficace, $\alpha$ trop grand le modèle n'apprend plus rien (tous les poids vont vers 0).


*  Typiquement, $\alpha$ peut-être choisi par une recherche en grille en essayant successivement `1e-3, 1e-4, 1e-5` (souvent on le prend très petit).

***A vous:*** Comparez avec la pénalisation `l1` (= lasso). Vous pourrez observer qu'elle impose à de très nombreux poids de devenir 0. On dit que les paramètres deviennent 'sparse'.

Cette sparsité est généralement recherchée pour des modèles linéaires: elle nous permet de retrouver les inputs qui sont vraiment important.

On utilise très rarement la pénalisation `l1` pour les réseaux de neurones à plusieurs couches. *texte en italique*

### Penalisation via le weight-decay


La pénalisation Ridge (ou l2) est aussi appelée weight decay. En voici la raison.

Considérons la fonction loss non-régularisée $loss(w)$. La descente de gradient (toute simple) correspond à changer les paramètres selon la règle suivante:
$$
w_i \leftarrow w_i - \ell \, \frac{\partial loss}{\partial w_i}
$$
où $\ell$ est le learning rate.  Considérons la loss régularisée:
$$
loss_{\alpha}(w)=loss(w) + \alpha \sum_i w_i^2
$$


Comment  s'écrit la descente de gradient maintenant ? Montrez que si les gradients initiaux sont `grads`, alors les gradients pénalisés sont:

    grads_penalized = [g + weight_decay * w for g, w in zip(grads, params)]


Ecrivez précisemment ♡ le lien entre `weight_decay` et $\alpha$.







In [ ]:
def jit_creator_with_weight_decay(hyper_param):


    model_init, model_apply = model_fnm(hyper_param)
    # dans adamw le 'w' c'est pour weight_decay
    optimizer = optax.adamw(1e-3, weight_decay=hyper_param["weight_decay"])


    @jax.jit
    def loss_compute(params, X,Y_true):
        Y_pred=model_apply(params,X)
        return binary_cross_entropy(Y_true,Y_pred)

    @jax.jit
    def accuracy_compute(params, X,Y_true):
        Y_pred=model_apply(params,X)
        return accuracy(Y_true,Y_pred)


    @jax.jit
    def update_model_param(optimizer_state, model_param, X,Y_true):
        loss,grads = jax.value_and_grad(loss_compute)(model_param, X,Y_true )

        #Il faut maintenant passer 'model_param' en argument:
        updates, optimizer_state = optimizer.update(grads, optimizer_state,model_param)
        #here the model_param is modified
        model_param = optax.apply_updates(model_param, updates)
        return loss,optimizer_state, model_param


    return model_init, model_apply, optimizer, loss_compute,accuracy_compute,update_model_param

Notez que optax n'utilise pas les biais pour le weight decay:

    optimizer = optax.adamw(1e-3, weight_decay=1e-5)   
    

est l'équivalent de

    optimizer = optax.adamw(1e-3, weight_decay=1e-5, mask=lambda p: jax.tree_map(lambda x: x.ndim > 1, p))



In [ ]:
folder="model_normal_with_weight_decay"

hyper_param={"learning_rate":1e-3,"batch_size":256,"n_layer":2,"dim_hidden":20, "weight_decay":2*1e-3}

losses,val_losses,val_steps=Agent.train(folder,hyper_param,jit_creator_with_weight_decay,10)

In [ ]:
plt.plot(losses,label="loss")
plt.plot(val_steps,val_losses,".",label="val_loss")
plt.legend();

### Recap

Pour récapituler, pour combattre le sur-apprentissage on peut:

* collecter plus de donnée, ou les enrichir
* réduire la capacité du modèle (le nombre de paramètre=poids)
* pénaliser les poids
* ajouter du dropout (on verra cela plus tard)


Des chercheurs ont aussi montré qu'une moindre précision dans les flotants (float32 au lieu de float64), introduisait un flou qui limiter le sur-apprentissage. On va même plus loin en randomisant les paramétres: c'est la théorie des réseaux de neurones bayesiens.


Cependant, il arrive que l'on cherche à faire du sur-apprentissage:

* Quand on veut tester si un modèle est assez gros, on peut vérifier qu'il a la capicité de sur-apprendre
* Quand on veut interpoler des données non-bruitées: données uniques (train=test).

Alors tout est permis pour faire diminuer au maximum la loss-train.


## Scores

Résumons:
* A partir des `x_test`, notre modèle prédit des proba `hat_y_test_proba`
* On prend un seuil de 0.5 puis on décide que `hat_y_test=(hat_y_test_proba>0.5)`
* On va maintenant mesurer de différentes façon l'écart entre  `hat_y_test` et le vrai `y_test`


En classification binaire, l'une des deux classes est appelée positive l'autre négative.

Traditionnellement, la classe positive est celle qui nous intéresse le plus ex: présence d'une maladie. Elle est souvent minoritaire. Ici, on décide que la classe positive c'est la classe 1: celle des critiques positives de film.



In [ ]:
folder="model_normal"
agent_result=Agent.load(folder)
Y_pred=agent_result.model_apply(agent_result.model_param,data["test"])
accuracy(labels["test"],Y_pred)

In [ ]:
Y_pred_cat=(Y_pred>0.5)[:,0].astype(int)
Y_pred_cat

In [ ]:
Y_true=labels["test"][:,0]
Y_true_cat=Y_true.astype(int)
Y_true_cat

###  Matrice de confusion

* les lignes indiques les vrai classes
* les colonnes les prédiction
* ex: l'intersection entre la première ligne et la première colonne indique le nombre de négatif bien classé.


In [ ]:
import sklearn.metrics
C=sklearn.metrics.confusion_matrix(Y_true_cat, Y_pred_cat)
"a dataframe just for presentation"
C_df=pd.DataFrame(data=C,columns=[ r"^-",r"^+"],index=[r"-",r"+"])
C_df

### Vrais/Faux positif/Negatif

On donne des noms à chacune des casses de la matrice de confusion:
$$
\begin{array}{c|cc}
& \hat - & \hat + \\
\hline
- & TN & FP \\
+ & FN &  TP   
\end{array}
$$

In [ ]:
TN=C[0,0]
FN=C[1,0]
FP=C[0,1]
TP=C[1,1];

***A vous:*** Faites les calculs demandé ci-dessous sans utiliser l'ordinateur:
* Indiquez  $(1\heartsuit)$  quel serait le résultat de  `confusion_matrix(y_test, y_test)`?  
* Indiquez  $(2\heartsuit)$  quel serait le résultat de  `confusion_matrix(y_test, y_rand)` avec `y_rand` un vecteur aléatoire de 0 et de 1?

### Précision/rappel


$$
\begin{align}
\text{precision} & =\frac{TP}{TP+FP}=  \frac{+\cap \hat +}{\hat +} = \text{accuracy of the positive predictions}\\
\text{recall} & =\frac{TP}{TP+FN} = \frac{+\cap \hat +}{ +} = \text{ratio of positive instances that are correctly detected}
\end{align}
$$

* Précision proche de 1: la plupart des prédictions son bonnes.
* Rappel proche de 1: on détecte la plupart des positifs

Si notre modèle détecte une maladie, il faut avoir un bon rappel. Surtout si on peut ensuite faire des examen supplémentaire pour éliminer les faux positifs.


In [ ]:
print("precision_score: %.2f"%(TP/(TP+FP)))
print("recall_score: %.2f"%(TP/(TP+FN)))

### F1 score

C'est la moyenne harmonique de la précision et du rappel:
$$
F_1= \frac{2}{\frac{1}{\text{precision}} + \frac{1}{\text{recall}}}
$$
 Le modèle a un bon F1-score quand précision et rappel sont tous les deux grands.






##  Changeons le seuil

On n'est pas obligé de prendre 0.5 comme seuil. Typiquement, on peut favoriser la classe minoritaire.

Avant de choisir un seuil, il est intéressant de regarder des courbes qui présente les résultats pour tous les seuils possibles.


Attention, certains modèle (ex: le gradient-stochastique de `sklearn`) ne renvoient pas des probas, mais simplement un score réel ex: entre -7000 et +9000. Mais le principe est le même: plus le score est grand et plus il faut classe la ligne dans les positifs. Ce qui suit reste valide: remplacer simplement l'intervalle [0,1] par l'intervalle [-7000,+9000].  



### Precision/Recall Tradeoff


In [ ]:
precisions, recalls, thresholds = sklearn.metrics.precision_recall_curve(Y_true, Y_pred)

def plot_precision_recall_vs_threshold(precisions, recalls, thresholds):
    plt.plot(thresholds, precisions[:-1], "b--", label="Precision", linewidth=2)
    plt.plot(thresholds, recalls[:-1], "g-", label="Recall", linewidth=2)
    plt.xlabel("Threshold", fontsize=16)
    plt.legend(loc="lower center", fontsize=16)
    plt.ylim([0, 1])

plt.figure(figsize=(8, 4))
plot_precision_recall_vs_threshold(precisions, recalls, thresholds)
plt.xlim([0, 1])

*   seuil  grand $\Rightarrow$ $\hat +$ petit $\Rightarrow$ grande précision et petit rappel.
*  Cas extrème:  Seuil $=1$  $\Rightarrow$ $\hat + = \emptyset$  $\Rightarrow$
$$
\begin{align}
\text{precision} & =  \frac{+\cap \hat +}{\hat +} = \frac 0 0 =  1 \\
\text{rappel}    & =  \frac{+\cap \hat +}{+}  = 0
\end{align}
$$


* Retenez: avec un grand seuil, on est très exigent pour classer les données +, donc je ne classe que des vrais +, donc grande précision.  




*   seuil  petit $\Rightarrow$ $\hat +$ grand $\Rightarrow$ petite précision et grand rappel.
*   Cas extrème:  Seuil=0 $\Rightarrow$ $\hat + = all$  $\Rightarrow$
$$
\begin{align}
\text{precision} & = \frac{+\cap \hat +}{\hat +} =  \frac{+}{all} = (\text{ ici } \frac 1 2 )  \\
\text{rappel}    & =  \frac{+\cap \hat +}{+}  =\frac{+}{+} =  1
\end{align}
$$
* retenez: avec un petit seuil,  on classe plein de donnée +, on ne va pas en rater beaucoup: grand rappel.







Vous pouvez choisir votre seuil en fonction de cette courbe, ou de la suivante:

In [ ]:
def plot_precision_vs_recall(precisions, recalls):
    plt.plot(recalls, precisions, "b", linewidth=2)
    plt.xlabel("Recall", fontsize=16)
    plt.ylabel("Precision", fontsize=16)
    plt.axis([0, 1, 0, 1])

plt.figure(figsize=(8, 6))
plot_precision_vs_recall(precisions, recalls)

Sur cette courbe, je choisirais le seuil qui correspond à un recall de 0.8: juste avant sa chutte.

### La courbe ROC

ROC signifie receiver operating characteristic (ROC). C'est une coubr très utilisée pour observer l'effet du seuillage:


* TPR = True Positive Rate = recall = ratio of positive instances that are correctly classified=
$$
\frac{+\cap \hat +}{ +}
$$
*  FPR = False Positive Rate =  ratio of negative instances that are incorrectly classified as positive =
$$
\frac{-\cap \hat +}{ - }
$$



In [ ]:
fpr, tpr, thresholds = sklearn.metrics.roc_curve(Y_true, Y_pred)
def plot_roc_curve(fpr, tpr, label=None):
    plt.plot(fpr, tpr, linewidth=2, label=label)
    plt.plot([0, 1], [0, 1], 'k--')
    plt.axis([0, 1, 0, 1])
    plt.xlabel('False Positive Rate', fontsize=16)
    plt.ylabel('True Positive Rate', fontsize=16)

plt.figure(figsize=(8, 6))
plot_roc_curve(fpr, tpr)


Le score AUC (area under the curve) c'est l'aire sous la courbe ROC. Au mieux elle vaut 1. Au pire 0.5.


In [ ]:
sklearn.metrics.roc_auc_score(Y_true, Y_pred)

###  courbe ROC à la main  $\flat$

Pour le plaisir, programmons notre propre courbe ROC.

In [ ]:
def TPR_FPR(threshold: float, y: np.ndarray, scores: np.ndarray):
    """
    Cette fonction renvoie:
        * le taux de vrais positifs : fractions des positifs qui sont classés positifs
        * le taux de faux  positifs : fractions des négatifs qui sont classés positifs.
    """


    """ on classe positifs (=en 1) les individus dont la proba estimée d'être 1 est > threshold   """
    y_hat = (scores >= threshold).astype(int)

    """Calculons les indexes des positifs et des négatifs. """
    index_P = (y == 1)
    index_N = (y == 0)


    TPR = np.sum(y_hat[index_P] == 1) / (np.sum(index_P)+1e-10)
    FPR = np.sum(y_hat[index_N] == 1) / (np.sum(index_N)+1e-10)

    return TPR, FPR

In [ ]:
thresholds=np.linspace(0,1,100)
a=[]
b=[]
for th in thresholds:
    fpr1, tpr1 =TPR_FPR(th,Y_true, Y_pred)
    a.append(fpr1)
    b.append(tpr1)
print(a)

plt.plot(b,a);

### Comparons avec les forêts aléatoires

On detaillera dans un prochain TP comment marche les forêts aléatoires.

Les arbres aléatoires de `sklearn` renvoie une proba avec la méthode méthode `predict_proba` (et pas `predict`)


In [ ]:
import sklearn.ensemble
forest_clf = sklearn.ensemble.RandomForestClassifier(random_state=42)
forest_clf.fit(data["train"],labels["train"])
Y_pred_forest=forest_clf.predict_proba(data["test"])

print(Y_pred_forest[:10,:])

Gardons uniquement la proba prédite pour la classe 1:

In [ ]:
Y_pred_forest=Y_pred_forest[:,1]

In [ ]:
fpr_forest, tpr_forest, thresholds_forest = sklearn.metrics.roc_curve(Y_true,Y_pred_forest)
plt.figure(figsize=(8, 6))
plot_roc_curve(fpr_forest, tpr_forest, "Random Forest")
plot_roc_curve(fpr, tpr, "Neural network")

plt.legend(loc="lower right", fontsize=16);

La forêt aléatoire est  moins bien.

* on n'a pas du tout réglé les hyper-paramètres de cette forêt.
*  les données textuelles ne sont pas favorable aux forêts aléatoires